In [105]:
import numpy as np
import glob
import random
import caffe

In [237]:
#init net
DATASET_DIR='../../dataset/cuhk01/campus/'
EXP_NAME='./py486/'
MODEL_FILE = EXP_NAME+'test.prototxt'
PRETRAINED = EXP_NAME+'snapshot/model_iter_40000.caffemodel'
ROOT='/home/liming/cluster/project/reid/train/cuhk01/'
EXP_NAME=ROOT+'orig_aug'
MODEL_FILE = EXP_NAME+'/test.prototxt'
PRETRAINED = EXP_NAME+'/snapshot/cuhk01_iter_48000.caffemodel' 

#'snapshot/model_iter_1000.caffemodel'
# PRETRAINED = '../../dataset/cuhk01/models/cuhk01_486_iter_100000.caffemodel'
model_name = EXP_NAME+PRETRAINED[PRETRAINED.rfind('_'):-11]
TEST_NUM=486
# TEST_NUM=100
caffe.set_device(0)
caffe.set_mode_gpu()
net = caffe.Classifier(MODEL_FILE, PRETRAINED,caffe.TEST)

In [238]:
def evaluateCMC(gtLabels,predictLists):
    N=len(gtLabels)
    R=len(predictLists[0])
    histogram=np.zeros(N)
    for testIdx in range(N):
        for rankIdx in range(R):
            histogram[rankIdx]+=1*(predictLists[testIdx][rankIdx]==gtLabels[testIdx])    #1*(true or false)=1 or 0
    cmc=np.cumsum(histogram)
    return cmc/N

In [239]:
N,C,H,W=net.blobs['data'].data.shape
h=0;w=0 #padding
transformer = caffe.io.Transformer({'data': (N,C,H+h*2,W+w*2)})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.array([ 104,  117,  123])) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

def readImages(images):
    imageLen=len(images)
    imageDataList=[]
    for imageIdx in range(imageLen):
        imageName=images[imageIdx]
        imageImage=transformer.preprocess('data', caffe.io.load_image(imageName))
        imageDataList.append(imageImage[:,h:h+H,w:w+W]) 
        imageIdx+=1
    #imageData and imageData
    imageData=np.asarray(imageDataList)
    return imageData

In [240]:
def readDir(DATA_DIR):
    test_num=TEST_NUM
    all_id={}
    file_lists=glob.glob(DATA_DIR+'*.png')
    for filename in file_lists:
        idx=filename.rfind('/')
        image_name=filename[idx+1:]
        person_id_str=image_name[:4]
        person_id = int(person_id_str)
        if all_id.has_key(person_id):
            all_id[person_id].append(filename)
        else:
            all_id[person_id]=[filename]

    id_names=all_id.keys()
    np.random.seed(0)
    train_idx_list=np.random.choice(id_names,len(id_names)-test_num,replace=False)

    test_idx_list=[]
    for x in id_names:
        if x not in train_idx_list:
            test_idx_list.append(x)
    print test_idx_list[:10]
    return test_idx_list, all_id

def readList(test_idx_list,all_id,DATA_DIR,rand_seed=0): 
    #print all_the_text
    probes=[]
    gallerys=[]
    #make sure random keeps seems for all experiments
    random.seed(rand_seed+10)
    for person_id in test_idx_list:
        probe_file_name=DATA_DIR+"%04d%03d.png"%(person_id,random.randint(1,2))
        gallery_file_name=DATA_DIR+"%04d%03d.png"%(person_id,random.randint(3,4))
        probes.append(probe_file_name)
        gallerys.append(gallery_file_name)
    return probes,gallerys

def calCMC(net,rand_times=10,DATA_DIR= '../../dataset/cuhk01/campus/'):
    cmc_list=[]
    rand_seeds=range(1,rand_times+1)
    test_idx_list, all_id=readDir(DATA_DIR)
    for i in range(rand_times):
        print 'Round %d with rand list:'%i
        probes,gallerys=readList(test_idx_list,all_id,DATA_DIR,rand_seeds[i])
        predictLists=generateScoreList(net,probes,gallerys)
        gtLabels=range(len(probes))
        cmc=evaluateCMC(gtLabels,predictLists)
        cmc_list.append(cmc)
        print cmc[0:5]
    return np.average(cmc_list,axis=0)

In [241]:
def generateScoreList(net,probes,gallerys):
    N,C,H,W=net.blobs['data'].data.shape
    from time import clock
    start=clock()
    #galleryData is same for each probe
    query_features=extract_features(probes)
    gallery_features=extract_features(gallerys)
    feature_time=clock()
    all_rank_list=rank_for_queries(query_features,gallery_features)
    rank_time=clock()
    print "feature_time:",(feature_time-start),"; rank_time:",(rank_time-start)
    return all_rank_list


def extract_features(file_list):
    file_len=len(file_list)
    features=[]
    batch_size=100
    round_times=file_len/batch_size+(1 if file_len%batch_size else 0)
    for batch_idx in range(round_times):
        cur_len=batch_size if batch_idx <file_len/batch_size else file_len%batch_size
        cur_list=file_list[batch_idx*batch_size+0:batch_idx*batch_size+cur_len]
        image_data=readImages(cur_list)
        net.blobs['data'].reshape(cur_len,C,H,W)
        net.blobs['data'].data[:] = image_data
        net.forward()
        normed_features=net.blobs['normed_feature'].data.copy()
        for idx in range(cur_len):
            cur_feature=np.squeeze(normed_features[idx,:])
            features.append(cur_feature)
    return features


def rank_for_queries(query_features,gallery_features):
    import numpy as np
    all_rank_list=[]
    for query_idx in range(len(query_features)):
        query_feature=query_features[query_idx]

        score_list=[]
        for gallery_idx in range(len(gallery_features)):
            gallery_feature=gallery_features[gallery_idx]
            dist = np.sqrt(np.sum((query_feature-gallery_feature)**2))
            similar_score=1.0/(1.0+dist)
            score_list.append(similar_score)
        #we get scoreList, then cal predictLists
        ranked_idx_list=np.argsort(score_list)[::-1]
        all_rank_list.append(ranked_idx_list)
    return all_rank_list


In [242]:
#caculate CMC
cmc_list=[]
cmc=calCMC(net,rand_times=14)
cmc_list.append(cmc)
cmc_all=np.average(cmc_list,axis=0)

[1, 4, 5, 8, 10, 12, 14, 17, 20, 24]
Round 0 with rand list:
feature_time: 3.103496 ; rank_time: 4.806782
[ 0.75720165  0.83127572  0.87860082  0.90123457  0.91152263]
Round 1 with rand list:
feature_time: 3.065403 ; rank_time: 4.778409
[ 0.74279835  0.82921811  0.85802469  0.88888889  0.90534979]
Round 2 with rand list:
feature_time: 3.057173 ; rank_time: 4.757918
[ 0.75720165  0.83127572  0.88683128  0.90740741  0.91975309]
Round 3 with rand list:
feature_time: 3.061629 ; rank_time: 4.785681
[ 0.74897119  0.82304527  0.85802469  0.88888889  0.90534979]
Round 4 with rand list:
feature_time: 3.060967 ; rank_time: 4.767766
[ 0.72633745  0.81893004  0.86213992  0.88683128  0.90740741]
Round 5 with rand list:
feature_time: 3.063925 ; rank_time: 4.772312
[ 0.72633745  0.81687243  0.84773663  0.8744856   0.88683128]
Round 6 with rand list:
feature_time: 3.062059 ; rank_time: 4.76514
[ 0.73045267  0.7962963   0.8600823   0.89506173  0.91152263]
Round 7 with rand list:
feature_time: 3.061169 

In [243]:
print 'cuhk01 test'+str(TEST_NUM)
print model_name+'; (H,W)='+str(H)+','+str(W)+'\n'
print('CMC from rank 1 to rank %d:'%(len(cmc_all)))
print(cmc_all[:50])
print('%.1f\t%.1f\t%.1f\t%.1f'%(100*cmc[0],100*cmc[4],100*cmc[9],100*cmc[19]))

cuhk01 test486
/home/liming/cluster/project/reid/train/cuhk01/orig_aug_48000; (H,W)=160,80

CMC from rank 1 to rank 486:
[ 0.73809524  0.81907701  0.86272781  0.88859494  0.90505585  0.91534392
  0.92489712  0.93268665  0.93885949  0.94312169  0.94664903  0.95047031
  0.95385068  0.95737801  0.9595826   0.96266902  0.96546149  0.96707819
  0.9691358   0.97045855  0.97281011  0.97413286  0.97545561  0.97663139
  0.97721928  0.97824809  0.97898295  0.9792769   0.97957084  0.98001176
  0.98045267  0.98089359  0.98206937  0.98236332  0.98295121  0.98309818
  0.98353909  0.98398001  0.9845679   0.98471487  0.98486185  0.98515579
  0.98544974  0.98574368  0.98647854  0.98647854  0.98662551  0.98691946
  0.98736038  0.98765432]
73.8	90.5	94.3	97.0


In [194]:
fds

NameError: name 'fds' is not defined

In [51]:
import random
class test():
    def __init__(self):
        self.all_id=[0,1,2,3,4,5]
        self.index=0
        self.batch_size=3
        self.id_dict={}
        for id in self.all_id:
            self.id_dict[id]=['id:%d'%(100*id)]*2
            
    def get_batch_list(self):
        count=0
        cur_range=xrange(self.index,self.index+self.batch_size)
        for idx in cur_range:
            if idx>=len(self.all_id):
                random.shuffle(self.all_id)
            self.index=idx%len(self.all_id)
            id=self.all_id[self.index]
            for file_name in self.id_dict[id][:self.max_per_id]:
                if count < self.batch_size:
                    count = count+1
                    yield (file_name,id)
                else:
                    break

    def info(self):
        batch_list=self.get_batch_list()
        for a,b in batch_list:
            print a,b

In [52]:
a=test()

In [ ]:
a.info()